# Getting Started

## Install Dependencies

This is a tutorial of using D4 in Python. Before you started trying this document, please make sure you have D4 package and `d4tools` binary installed. 

* To install the d4tools binary, please read the instruction from [this link](https://github.com/38/d4-format#installation--2-minutes)
* To install `pyd4`, please run command `pip install pyd4`

## Download dataset

To download the dataset we are using for this interactive document, please use the following links:
* WGS Sample HG002: https://home.chpc.utah.edu/~u0875014/hg002.cram
* Reference Genome: https://home.chpc.utah.edu/~u0875014/hg19.fa.gz

Please make sure that index files is accessible for both CRAM file and reference genome.


# Download Data

In [5]:
import os
os.system("mkdir -p data")
os.system("cd data && wget --continue https://home.chpc.utah.edu/~u0875014/hg002.cram")
os.system("cd data && wget --continue https://home.chpc.utah.edu/~u0875014/hg002.cram.crai")
os.system("cd data && wget --continue https://home.chpc.utah.edu/~u0875014/hg19.fa.gz")
os.system("cd data && wget --continue https://home.chpc.utah.edu/~u0875014/hg19.fa.gz.fai")

0

# Create depth profile for an alignment file